<a href="https://colab.research.google.com/github/edufantini/music-gen/blob/main/src/MusicGeneratorLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Music Generator with LSTM

##About

Music resamples language as a temporal sequence of articulated sounds. They say something, often something human.

Although, there are crucial differences between language and music. We can still describe it as a sequence of symbols in the simplest form of understanding. Translating something complex into something simpler, but usable by computational models.

Thus, the objective of this project is to establish a communication between the human, that understands music in the most intense way that the brain can interpret through information, and the machine.

We'll create a model that can generate music based on the input information, i.e., generate a sequence of sounds which are related in some way with the sounds passed as input.

We'll use Natural Language Processing (NLP) methods, observing the music as it were a language, abstracting it. Doing this, the machine can recognize and process similar data.

On the first step, we'll use text generation techniques, using Recurrent Neural Networks (RNNs) and Long-Short Term Memories (LSTMs). With the effectiveness of the training, even if it's reasonable, we'll perform the same implementation using specific methods such as Attention.



## Imports

In [44]:
# Basic libraries
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn.functional as F

# Preprocessing data libraries
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

# Model libraries
import torch
import torch.nn as nn
import torch.optim as optim

# Data visualization
from torch.utils.tensorboard import SummaryWriter
from tqdm.notebook import tqdm #for loading bars

In [2]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

In [3]:
!git clone https://github.com/edufantini/music-gen.git

Cloning into 'music-gen'...
remote: Enumerating objects: 19522, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 19522 (delta 19), reused 43 (delta 14), pack-reused 19468
Receiving objects: 100% (19522/19522), 221.29 MiB | 23.34 MiB/s, done.
Resolving deltas: 100% (89/89), done.


In [4]:
from music_gen.src.GetData import *

## Dataset

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [6]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"

In [7]:
%cd /content/gdrive/My Drive/Kaggle

/content/gdrive/My Drive/Kaggle


In [8]:
!kaggle datasets download -d edufantini/songs-in-midi

 99% 224M/226M [00:02<00:00, 88.6MB/s]
100% 226M/226M [00:02<00:00, 82.7MB/s]


In [9]:
!ls

clean_midi  dataset  kaggle.json  music-gen  songs-in-midi.zip


In [10]:
!unzip \*.zip  && rm *.zip

Archive:  songs-in-midi.zip
replace clean_midi/.38 Special/Caught Up In You.mid? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [11]:
path = '/content/gdrive/My Drive/Kaggle/clean_midi/AC_DC/'

dataset = []

for filename in os.listdir(path):
  if filename.endswith("mid"): 
    # Your code comes here such as 
    print(path + filename)
    #if filename is not 'Back_In_Black.mid':
    data = encode_data(path+filename, 32)
    dataset.append(data)

/content/gdrive/My Drive/Kaggle/clean_midi/AC_DC/Dirty_Deeds_Done_Dirt_Cheap.mid
Processing file /content/gdrive/My Drive/Kaggle/clean_midi/AC_DC/Dirty_Deeds_Done_Dirt_Cheap.mid
Processing part 1/3


Converting measures from part 1: 100%|###############################################################################| 58/58 [00:00<00:00, 272.90it/s]


Processing part 2/3


Converting measures from part 2: 100%|##############################################################################| 63/63 [00:00<00:00, 2566.00it/s]



Processing part 3/3


...

Converting measures from part 6: 100%|###############################################################################| 55/55 [00:00<00:00, 346.75it/s]



Processing part 7/7


Converting measures from part 7:  46%|####################################3                                          | 35/76 [00:00<00:00, 179.45it/s]

StringInstrument


Converting measures from part 7: 100%|###############################################################################| 76/76 [00:00<00:00, 241.36it/s]


In [12]:
len(dataset)

16

## Preprocess data

```preprocess_bar(encoded_seq, n_in=32, n_out=32)``` uma barra (32 frames) codificada em multi-hot pre-processa essa barra de forma a gerar os valores splitados em X e y

In [13]:
def preprocess_bar(encoded_seq, n_in=32, n_out=32):
  # create lag copies of the sequence
  df = pd.DataFrame(encoded_seq)
  df = pd.concat([df.shift(n_in-i-1) for i in range(n_in)], axis=1)
  # drop rows with missing values
  df.dropna(inplace=True)
  # specify comumns for inout and output values
  values = df.values
  width = encoded_seq.shape[1]
  X = values[:, 0:width*(n_in-1)].reshape(n_in-1, width)
  y = values[:, width:].reshape(n_in-1, width)
  return X,y

```create_dataloader(dataset, batch_size=1)``` converte um dataset com n musicas em um dataloader. 

In [14]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

def create_dataloader(dataset, batch_size=1):
  X = []
  y = []
  n_songs = 0
  n_parts= 0
  n_bars= 0

  # create a two arrays X, y with bars
  for song in dataset:
    for part in song:
      for bar in part:
        xa, ya = preprocess_bar(bar)
        X.append(xa)
        y.append(ya)

  X = np.array(X)
  y = np.array(y)
  X = torch.from_numpy(X)
  y = torch.from_numpy(y)
  print(X.shape, y.shape)
  train_ds = TensorDataset(X, y)
  train_dl = DataLoader(train_ds, batch_size=1, shuffle=False)

  return train_dl
           

O dataloader é dividido em duas partes:

  1.   context
  2.   target

onde context são os valores que serão passados como entrada para o modelo - neste caso, esses valores serão 31 frames localizados em cada uma das 5010 barras das musicas e cada frame possui 88 notas.


In [15]:
train_dl = create_dataloader(dataset)

torch.Size([5010, 31, 88]) torch.Size([5010, 31, 88])


In [60]:
train_dl.dataset.tensors[0][200]

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], dtype=torch.float64)

In [16]:
def create_vocab(dataset):
  vocab = []
  for song in dataset:
    for part in song:
      for bar in part:
        vocab.append(bar)

  vocab = np.array(vocab)
  vocab = vocab.reshape(vocab.shape[0]*vocab.shape[1], vocab.shape[2])
  vocab = np.unique(vocab, axis=0)
  
  print(vocab)
  
  return vocab

Temos 837 frames distintos

In [17]:
diff_frames = create_vocab(dataset)
print('\nvocab len: {}'.format(diff_frames.shape[0]))

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]

vocab len: 626


## Model

Some important definitions

In [ ]:
n_frames_input = 31
n_frames_output = 31
n_bars_input = len(train_dl.dataset.tensors[0]) # number of rows of the dataloader
bar_len = 31 # how many frames it's gonna take in a timestep
num_layers = 31
frame_len = 88
hidden_size = 88
num_epochs = 3
batch_size = 1
lr = 0.003
print('Number of bars in the input dataset: {}'.format(n_bars_input))

In [45]:
class RNN(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, output_size):
    super(RNN, self).__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers

    #self.embed = nn.Embedding(input_size, hidden_size)
    self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=False)
    self.fc = nn.Linear(hidden_size, output_size)
    self.act = nn.Hardsigmoid()

  def forward(self, x, hidden, cell):

    # Passing in the input and hidden state into the model and obtaining outputs
    out, (hidden, cell) = self.lstm(x.unsqueeze(1), (hidden, cell))

    # Reshaping the outputs such that it can be fit into the fully connected layer
    out = self.fc(out.contiguous().view(-1, self.hidden_size))
    out = self.act(out)
    
    return out, (hidden, cell)

  def init_hidden(self, batch_size):
    # This method generates the first hidden state of zeros which we'll use in the forward pass
    # We'll send the tensor holding the hidden state to the device we specified earlier as well
    hidden = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)
    cell = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)
    return hidden, cell

In [83]:
# Instantiate the model with hyperparameters
# We'll also set the model to the device that we defined earlier (default is CPU)
model = RNN(input_size=frame_len,
                   output_size=frame_len,
                   hidden_size=hidden_size,
                   num_layers=num_layers).to(device)

## Train

In [84]:
# converts one frame into torch tensor
def multi_hot_tensor(frame):
  tensor = torch.from_numpy(frame)
  return tensor

In [85]:
# retrieve data from dataloader
def get_sample(dataloader):

  input = torch.zeros(n_bars_input, bar_len, frame_len)
  target = torch.zeros(n_bars_input, bar_len, frame_len)

  for sample, (xb, yb) in enumerate(dataloader): # gets the samples
    input[sample] = xb
    target[sample] = yb
  
  return input, target

In [89]:
def train(model, optimizer, loss_fn, dataloader, batch_size=1, num_epochs=3):

  print("\nStarting training...")

  for epoch in range(1, num_epochs + 1):
    training_loss = 0.0

    print('> EPOCH #', epoch)

    input, target = get_sample(dataloader)
    input = input.to(device)
    target = target.to(device)

    for bar in tqdm(range(n_bars_input)):
      # Initialize hidden and cells
      hidden, cell = model.init_hidden(batch_size)

      # Generate predictions
      output, (hidden, cell) = model(input[bar,:], hidden, cell)

      # Compute the loss and backpropag         
      loss_step = loss_fn(output, target[bar, :])
      loss_step.backward() # Does backpropagation and calculates gradients
      optimizer.step() # Updates the weights accordingly
      optimizer.zero_grad() # Clears existing gradients from previous frame
      
      training_loss += loss_step.item()
    
    training_loss /= len(train_dl.dataset)
      
    if epoch%1 == 0:
      print('Epoch: {}/{}.............'.format(epoch, num_epochs), end=' ')
      print("Loss: {:.4f}".format(training_loss))

In [90]:
optimizer = optim.Adam(model.parameters(), lr=lr)
loss_fn = nn.BCELoss()
train(model, optimizer, loss_fn, train_dl)


Starting training...
> EPOCH # 1



Epoch: 1/3............. Loss: 0.8174
> EPOCH # 2



Epoch: 2/3............. Loss: 0.8162
> EPOCH # 3



Epoch: 3/3............. Loss: 0.8162


## Test

In [112]:
def generate(model, initial_bar, predict_len=31, batch_size=1, temperature=0.85):
  output = []
  hidden, cell = model.init_hidden(batch_size)
  initial_bar = initial_bar.to(device)
  for _ in range(predict_len):
    out, (hidden, cell) = model(initial_bar, hidden, cell)
    output.append(out)
  
  return output

In [119]:
initial_bar = train_dl.dataset.tensors[0][3847]
initial_bar = initial_bar.to(torch.float)
print(f'Input: ')
torch.set_printoptions(threshold=10_000)
print(initial_bar)

output = generate(model, initial_bar, predict_len=1)
print(f'Output: ')
print(output)

Input: 
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0